# Lab 1. Bag of Words (BoW)

### Import libraries

In [1]:
from typing import List

from bow import BoW
from dataset import Dataset
from image_classifier import ImageClassifier

c:\Users\marti\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load helper functions

Una palabra aquí es un vector de 128 elementos.

Son clusters de kas caracreristicas que quiero sacar. 

Para reducir dimensionalidad, una opcion sería hacer un PCA. En otro caso, podriamos agrupar

Es mejor ampliar el numero de palabras (size) que el numero de iteraciones. Porque kmeans empieza en aleaotorio. Cada iteracion obtenemos una salida.

In [2]:
def build_vocabulary(dataset: List[str], vocabulary: str = 'vocabulary', feature_type: str = 'SIFT', size: int = 100, iterations: int = 20):
    """Build a vocabulary.

    Args:
        dataset: Paths to the training images.
        vocabulary: Relative path to the file (without extension) where the vocabulary will be saved.
        feature_type: Feature extractor { SIFT, KAZE }.
        size: Number of words in the vocabulary. -- Cuántos clusters quiero sacar
        iterations: Maximum number of K-means iterations. -- Num iteracionses que va a hacer el modelo

    """
    bow = BoW()
    bow.build_vocabulary(dataset, feature_type=feature_type, vocabulary_size=size, iterations=iterations)
    bow.save_vocabulary(vocabulary)

def train_classifier(dataset: List[str], vocabulary: str = 'vocabulary', classifier: str = 'classifier', iterations: int = 100, kernel: str = 'SVM_LINEAR'):
    """Train an SVM classifier.

    Args:
        dataset: Paths to the training images.
        vocabulary: Relative path to the vocabulary file (without extension).
        classifier: Relative path to the file (without extension) where the classifier will be saved.
        iterations: Maximum number of SVM iterations.

    """
    bow = BoW()
    bow.load_vocabulary(vocabulary)
    print("Kernel elegido: ", kernel)
    image_classifier = ImageClassifier(bow)
    image_classifier.train(dataset, iterations=iterations, kernel_type=kernel)
    image_classifier.save(classifier)

def predict(dataset: List[str], dataset_name: str = "", vocabulary: str = 'vocabulary', classifier: str = 'classifier'):
    """Perform inference on a dataset.

    Args:
        dataset: Paths to the images.
        dataset_name: Dataset descriptive name.
        vocabulary: Relative path to the vocabulary file (without extension).
        classifier: Relative path to the classifier file (without extension).

    """
    bow = BoW()
    bow.load_vocabulary(vocabulary)

    image_classifier = ImageClassifier(bow)
    image_classifier.load(classifier)
    image_classifier.predict(dataset, dataset_name=dataset_name)

### Load datasets

In [3]:
training_set = Dataset.load('../dataset/training', '*.jpg')
validation_set = Dataset.load('../dataset/validation', '*.jpg')

### Build vocabulary and train a SVM classifier

In [5]:
build_vocabulary(training_set) # extraccion características
train_classifier(training_set) # entrena svm con las caracteristicas anteriores


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [01:07<00:00, 44.00image/s]

Clustering descriptors into 100 words using K-means...


TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:05<00:00, 45.88image/s]

Training SVM...


## Modificación 1: Cambio del algoritmo SIFT por KAZE
Es un algoritmo de deteccion y descripcion de características de imagenes.
KAZE es una extension de SURF

In [5]:
build_vocabulary(training_set, vocabulary="vocabulary_KAZE", feature_type="KAZE", size=100, iterations=50) # extraccion características


BUILDING DICTIONARY

Computing KAZE descriptors...
100%|██████████| 2985/2985 [09:13<00:00,  5.39image/s]

Clustering descriptors into 100 words using K-means...


In [4]:
train_classifier(training_set, vocabulary="vocabulary_KAZE", classifier="classifier_KAZE", kernel= "SVM_RBF") # entrena svm con las caracteristicas anteriores

Kernel elegido:  SVM_RBF


TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [16:19<00:00,  3.05image/s]

Training SVM...


In [6]:
predict(training_set, "Training", vocabulary="vocabulary_KAZE", classifier="classifier_KAZE")



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             38.0    1.0     2.0      0.0        11.0          8.0      1.0         23.0       4.0     4.0           2.0    2.0     2.0     1.0           17.0
Coast                2.0  168.0    11.0      8.0         5.0          2.0      0.0          4.0      15.0     3.0          35.0    0.0     0.0     0.0            7.0
Forest               1.0    0.0   178.0      0.0         3.0          1.0      0.0          1.0      17.0     0.0           8.0   11.0     6.0     0.0            2.0
Highway              0.0   21.0     0.0     89.0         8.0          3.0      0.0          2.0       8.0     0.0          12.0    0.0     7.0     3.0            7.0
Industrial           3.0    4.0     7.0      7.0        80.0         10.0      0.0          9.0      16.0     1.0  

In [7]:
predict(validation_set, "Validation", vocabulary="vocabulary_KAZE", classifier="classifier_KAZE")



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             24.0    1.0     5.0      1.0         4.0          5.0      2.0         33.0       6.0     2.0           0.0    4.0     4.0     3.0            6.0
Coast                0.0   61.0     3.0      3.0         1.0          1.0      0.0          2.0       8.0     0.0          16.0    0.0     0.0     0.0            3.0
Forest               0.0    1.0    70.0      0.0         0.0          0.0      0.0          1.0       7.0     0.0           8.0   10.0     1.0     1.0            1.0
Highway              0.0   22.0     0.0     45.0         5.0          5.0      0.0          1.0       7.0     1.0           9.0    0.0     2.0     1.0            2.0
Industrial           3.0    3.0     1.0      5.0        31.0          8.0      0.0          5.0       8.0     0.0

In [ ]:
import pickle
voc = pickle.load(open("vocabulary_KAZE.pickle", "rb"))

### Perform inference on the training set

Matriz de confision, dónde se está confundiendo en el conjunto de entrenamiento. Por ejemplo en campo abierto y playa

Esta técnica va a sobreentrenar mucho, porque tiene limitaciones.

In [6]:
predict(training_set, "Training")



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             25.0    1.0     1.0      0.0        17.0          5.0      1.0         31.0       2.0     3.0           9.0   10.0     3.0     2.0            6.0
Coast                2.0  181.0     8.0      8.0         4.0          1.0      0.0          0.0      20.0     0.0          20.0    0.0     2.0     6.0            8.0
Forest               0.0    0.0   205.0      0.0         2.0          0.0      0.0          0.0       9.0     0.0          10.0    1.0     1.0     0.0            0.0
Highway              0.0   32.0     0.0     65.0        10.0          5.0      1.0          1.0       9.0     0.0           7.0    3.0     8.0     8.0           11.0
Industrial           4.0    5.0     0.0      3.0       103.0         14.0      2.0          9.0      11.0     3.0  

### Perform inference on the validation set

In [7]:
predict(validation_set, "Validation")



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             18.0    5.0     2.0      2.0        10.0          6.0      1.0         25.0       2.0     8.0           4.0    5.0     4.0     1.0            7.0
Coast                0.0   66.0     1.0      5.0         1.0          1.0      0.0          0.0      13.0     0.0           8.0    1.0     0.0     2.0            2.0
Forest               0.0    0.0    86.0      0.0         0.0          0.0      0.0          0.0       4.0     0.0           7.0    1.0     1.0     1.0            0.0
Highway              1.0   27.0     0.0     36.0         7.0          3.0      0.0          0.0       8.0     0.0           3.0    1.0     4.0     5.0            5.0
Industrial           4.0    2.0     0.0      3.0        34.0         17.0      1.0          4.0       9.0     0.0

In [9]:
import pickle
voc = pickle.load(open("vocabulary.pickle", "rb"))